# Agentic AI MCP - Client Only

This notebook connects to a remote MCP server and runs agentic workflows.
The MCP server should be running on another machine (see `server.ipynb`).


## Step 1: Setup

In [1]:
# install if needed
# !uv pip install agentic-ai-mcp==0.5.3

import agentic_ai_mcp

print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.5.3


## Step 2: Make sure you have a valid API_KEY

The LLM runs on this machine, so you need the API key here.
- For **Anthropic** (default): Set `ANTHROPIC_API_KEY`
- For **OpenAI**: Set `OPENAI_API_KEY`

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

# Check for API keys
if os.getenv("ANTHROPIC_API_KEY"):
    print("ANTHROPIC_API_KEY is set")
else:
    print("WARNING: ANTHROPIC_API_KEY not found")

if os.getenv("OPENAI_API_KEY"):
    print("OPENAI_API_KEY is set")
else:
    print("INFO: OPENAI_API_KEY not found (optional, only needed for OpenAI provider)")

ANTHROPIC_API_KEY is set
OPENAI_API_KEY is set


## Step 3: Connect to Remote MCP Server

Use the `mcp_url` parameter to connect to an existing MCP server.

In [3]:
from agentic_ai_mcp import AgenticAI

# connect to remote MCP server (change this to your server's URL)
MCP_SERVER_URL = "http://127.0.0.1:8888/mcp"  # change to remote IP if needed

ai = AgenticAI(mcp_url=MCP_SERVER_URL, verbose=True)  # client-only mode: connect to existing server

print(f"Client configured to connect to: {MCP_SERVER_URL}")

Client configured to connect to: http://127.0.0.1:8888/mcp


## Step 4a: Run Agent (Simple Task)

In [4]:
result = await ai.run("calculate 2 + 2, then greet 'tom' the result number of times")
print()
print(">>>")
print()
print(result)

Loading tools from: http://127.0.0.1:8888/mcp
Loaded tools: ['add', 'multiply', 'greet', 'grid_scan', 'line_scan', 'raster_scan', 'set_tip_position', 'set_scan_parameters', 'get_force_curve', 'approach_surface', 'retract_tip', 'get_scan_image', 'calibrate_scanner']

PROMPT: calculate 2 + 2, then greet 'tom' the result number of times



/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/agentic.py:185: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self._agent = create_react_agent(


STEP 1: add({'a': 2, 'b': 2})
  → CallToolResult(content=[TextContent(type='text', text='{"result":4}', annotations=None, meta=None)], structured_content={'result': 4}, meta=None, data={'result': 4}, is_error=False)

STEP 2: greet({'name': 'tom', 'times': 4})
  → CallToolResult(content=[TextContent(type='text', text='{"result":"Hello, tom! Hello, tom! Hello, tom! Hello, tom! "}', annotations=None, meta=None)], structured_content={'result': 'Hello, tom! Hello, tom! Hello, tom! Hello, tom! '}, meta=None, data={'result': 'Hello, tom! Hello, tom! Hello, tom! Hello, tom! '}, is_error=False)

RESULT: Perfect! 2 + 2 = 4, and Tom has been greeted 4 times: **Hello, tom! Hello, tom! Hello, tom! Hello, tom!**


>>>

Perfect! 2 + 2 = 4, and Tom has been greeted 4 times: **Hello, tom! Hello, tom! Hello, tom! Hello, tom!**


## Step 4b: Run Agent (Complex Task)

In [5]:
result = await ai.run_with_planning(
    "calculate (((0+1)+(1+0))+1), then greet 'Alice' the result number of times"
)
print()
print(">>>")
print()
print(result)


PLANNING MODE
TASK: calculate (((0+1)+(1+0))+1), then greet 'Alice' the result number of times

PLAN:
  1. Use add(a=0, b=1) to calculate the first inner sum
  2. Use add(a=1, b=0) to calculate the second inner sum
  3. Use add(a=result from step 1, b=result from step 2) to add those two results
  4. Use add(a=result from step 3, b=1) to add 1 to get the final result
  5. Use greet(name="Alice", times=result from step 4) to greet Alice that many times

EXECUTING STEP 1/5: Use add(a=0, b=1) to calculate the first inner sum
----------------------------------------


/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/workflows/planning.py:149: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  step_agent = create_react_agent(llm, tools)


  TOOL CALL 1: add({'a': 0, 'b': 1})
    → CallToolResult(content=[TextContent(type='text', text='{"result":1}', annotations=None, meta=None)], structured_content={'result': 1}, meta=None, data={'result': 1}, is_error=False)
  AI RESPONSE: The first inner sum is **1**. 

The add function returned 0 + 1 = 1.
----------------------------------------
STEP 1 COMPLETE

EXECUTING STEP 2/5: Use add(a=1, b=0) to calculate the second inner sum
----------------------------------------
  TOOL CALL 1: add({'a': 1, 'b': 0})
    → CallToolResult(content=[TextContent(type='text', text='{"result":1}', annotations=None, meta=None)], structured_content={'result': 1}, meta=None, data={'result': 1}, is_error=False)
  AI RESPONSE: Step 2: The second inner sum is **1**.

The add function returned 1 + 0 = 1.
----------------------------------------
STEP 2 COMPLETE

EXECUTING STEP 3/5: Use add(a=result from step 1, b=result from step 2) to add those two results
----------------------------------------
  TOOL 

In [6]:
result = await ai.run_with_planning("""
You are controlling an AFM (Atomic Force Microscope). Please complete the following full experiment sequence step by step, using ALL available tools:

1. First, calibrate the scanner using 'xy' calibration mode
2. Set the scan parameters with setpoint=0.5, p_gain=1.2, i_gain=0.6, scan_rate=0.8
3. Move the tip to starting position x=10.0, y=10.0, z=0.0
4. Approach the surface with target_setpoint=0.5 and max_z_travel=15.0
5. Perform a grid scan from x_start=0.0 to x_end=50.0 and y_start=0.0 to y_end=50.0 with step_size=1.0 and scan_speed=0.5
6. Perform a raster scan with width=50.0, height=50.0, resolution=512, scan_angle=0.0, scan_speed=0.5
7. Perform a line scan from x_start=0.0 to x_end=50.0 at y_position=25.0 with num_points=256
8. Acquire a force curve at x=25.0, y=25.0 with z_start=100.0, z_end=-10.0, num_points=512
9. Retrieve the scan image for the 'height' channel
10. Add the grid scan points (100) and line scan points (256) together to get total points scanned
11. Multiply the total points by 2 to get the oversampled count
12. Greet the experiment results with name='AFM Experiment Complete' and times=2
13. Finally, retract the tip by 75.0 nm

Report the results of each step clearly.
""")

print(result)


PLANNING MODE
TASK: 
You are controlling an AFM (Atomic Force Microscope). Please complete the following full experiment sequence step by step, using ALL available tools:

1. First, calibrate the scanner using 'xy' calibration mode
2. Set the scan parameters with setpoint=0.5, p_gain=1.2, i_gain=0.6, scan_rate=0.8
3. Move the tip to starting position x=10.0, y=10.0, z=0.0
4. Approach the surface with target_setpoint=0.5 and max_z_travel=15.0
5. Perform a grid scan from x_start=0.0 to x_end=50.0 and y_start=0.0 to y_end=50.0 with step_size=1.0 and scan_speed=0.5
6. Perform a raster scan with width=50.0, height=50.0, resolution=512, scan_angle=0.0, scan_speed=0.5
7. Perform a line scan from x_start=0.0 to x_end=50.0 at y_position=25.0 with num_points=256
8. Acquire a force curve at x=25.0, y=25.0 with z_start=100.0, z_end=-10.0, num_points=512
9. Retrieve the scan image for the 'height' channel
10. Add the grid scan points (100) and line scan points (256) together to get total points sc

## Step 5: Using OpenAI Provider (Optional)

You can use OpenAI instead of Anthropic by specifying the `provider` parameter.

**Note:** Requires `OPENAI_API_KEY` to be set in your `.env` file.

In [8]:
# Using OpenAI provider
# Uncomment to use OpenAI instead of Anthropic

from agentic_ai_mcp import AgenticAI

ai_openai = AgenticAI(
    mcp_url=MCP_SERVER_URL,
    provider="openai",
    model="gpt-4o-mini",  # or "gpt-4o", "gpt-4-turbo", etc.
    verbose=True
)

result = await ai_openai.run("calculate 2 + 2")
print(result)

Loading tools from: http://127.0.0.1:8888/mcp
Loaded tools: ['add', 'multiply', 'greet', 'grid_scan', 'line_scan', 'raster_scan', 'set_tip_position', 'set_scan_parameters', 'get_force_curve', 'approach_surface', 'retract_tip', 'get_scan_image', 'calibrate_scanner']

PROMPT: calculate 2 + 2

STEP 1: add({'a': 2, 'b': 2})
  → CallToolResult(content=[TextContent(type='text', text='{"result":4}', annotations=None, meta=None)], structured_content={'result': 4}, meta=None, data={'result': 4}, is_error=False)

RESULT: The result of \( 2 + 2 \) is \( 4 \).

The result of \( 2 + 2 \) is \( 4 \).


## Step 6: Using Custom Settings (Optional)

You can customize retry behavior and other settings using the `Settings` class.

In [14]:
# Using custom settings
# Uncomment to customize retry behavior

from agentic_ai_mcp import AgenticAI, Settings

settings = Settings(
    max_retries=10,           # More retries for unstable connections
    retry_base_delay=2.0,     # Longer initial delay
    retry_max_delay=120.0,    # Longer max delay
)

ai_custom = AgenticAI(
    mcp_url=MCP_SERVER_URL,
    settings=settings,
    verbose=True
)

result = await ai_custom.run("calculate 2 + 2")
print(result)

Loading tools from: http://127.0.0.1:8888/mcp
Loaded tools: ['add', 'multiply', 'greet', 'grid_scan', 'line_scan', 'raster_scan', 'set_tip_position', 'set_scan_parameters', 'get_force_curve', 'approach_surface', 'retract_tip', 'get_scan_image', 'calibrate_scanner']

PROMPT: calculate 2 + 2



/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/agentic.py:185: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self._agent = create_react_agent(


STEP 1: add({'a': 2, 'b': 2})
  → CallToolResult(content=[TextContent(type='text', text='{"result":4}', annotations=None, meta=None)], structured_content={'result': 4}, meta=None, data={'result': 4}, is_error=False)

RESULT: The result of 2 + 2 is **4**.

The result of 2 + 2 is **4**.
